# DataScience with Spatial Computing

**Explore your data in Augmented and Virtual Reality using PlotAR**

> **WARNING!**
> 
> PlotAR is an alpha versioned package and is published as-is and without any guarantees. This means: APIs can change, sometimes it is instable, and security is not a priority at this moment. This also means, that your data could be accessed by others who can access your server.
> 
> **We do not recommend to use this for sensitive data at the moment**

Welcome to the PlotAR Hands-On notebook.

In this notebook we have first some curated examples for the workshop. In the second part there are many more examples we have done in the past. You can try them out as you go along. Open the Table of Contents pane to the left (where by default the file browser is) for an overview.

If you like PlotAR consider leaving a star at <https://www.github.com/thomann/plotAR>.

## Imports

In [ ]:
import plotar
import pandas as pd
import numpy as np
from pathlib import Path
import requests
import json

For comparisons with 2d plots:

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

Some helper stuff:

In [ ]:
from IPython.display import HTML, JSON, display
from dotenv import load_dotenv

In [ ]:
load_dotenv()

## Canton of Zurich Relief

Let us start with some geo-spatial data that we prepared. The somewhat involved details of the data preparation using geopandas etc. can be seen in [this Notebook Gist](https://gist.github.com/thomann/7c74e72f003eedfb0823b054f1f021f6#file-flights-ipynb).

In [ ]:
with np.load('data/canton_zrh.npz', ) as canton_zrh:
    dhm=canton_zrh['dhm']
    landsat=canton_zrh['landsat']
    dhm_lon_lv95 = canton_zrh['dhm_lon_lv95']
    dhm_lat_lv95 = canton_zrh['dhm_lat_lv95']
    center = canton_zrh['center']
    ground = canton_zrh['ground']
f"{dhm.shape=} {landsat.shape=} {dhm_lon_lv95.shape=} {dhm_lat_lv95.shape=}"

These datasets are:
* `dhm`: the Digital Height Model
* `landsat`: satellite image
* `dhm_lon_lv95` and `dhm_lat_lv95`: the respective longitude and latitude values of the grid - in [LV95 System](https://www.swisstopo.admin.ch/de/schweizer-koordinatensystem), that is in meters.
* `center`: the coordinate of the center of this grid (in lv95)
* `ground`: the lowest altitude of this grid (in meter)

In [ ]:
f"{center=} {ground=}"

So let's look at the height model data, this is in meter:

In [ ]:
px.imshow(dhm)

And the satellite image (in RGB) - to reduce the image size we only show every 10th pixel:

In [ ]:
px.imshow(landsat[::10,::10,:])

All values now are in meter, we do not want PlotAR to squeeze everything into a box, so we will do the scaling ourselves. Some testing showed that a scale of 1:40'000 looks good. Also we want to overstate the height by a factor of 5 to see it better.

Also we decrease the satellite image for network and performance reason - keep the factor a smaller integer for more details.

You can play around with different values and look what happens.

In [ ]:
SCALE = 1 / 40000
HEIGHT_FACTOR = 5

IMAGE_DECREASE_BY_FACTOR = 5 ## has to be an integer >= 1
surfacecolor = landsat[::IMAGE_DECREASE_BY_FACTOR,::IMAGE_DECREASE_BY_FACTOR,:]

# make sure the bottom of the heigts is at -1
heights = dhm - ground

Now let's create our first immersive plot with PlotAR's surfacevr function.
For that we need:
* first argument is the grid/matrix of heights. We have to scale this height to be consistent with the other parameters.
* `x` and `y` give the coordinates along the edges of the grid. We want them to be centered around `(0,0)` and scaled as well.
* `surfacecolor` can be any image, it will be projected on the surface, here we use a downsampled version of the landsat image.
* Lastly we specify a `name` that will be part of the plot.

Now plot the surface and save it in the `fig`-variable:

In [ ]:
fig = plotar.surfacevr(
    heights * SCALE * HEIGHT_FACTOR,
    x=(dhm_lon_lv95 - center[0]) * SCALE,
    y=(dhm_lat_lv95 - center[1]) * SCALE,
    surfacecolor=surfacecolor,
    auto_scale=False,
    name="Zurich Landsat/Height",
)
# print the fig to see it in here
fig

First you can use your mouse to zoom and rotate this plot as usual.

Then however go and scan the QR code with your Tablet of Smartphone to get to the same page on that device.

**Note:** Only Safari and Chrome work - Firefox mobile did not work in our tests (neither on iOS nor on Android). Furthermore in iOS rather use the Camera App to scan the QR code and not the QR-Scanning App (you then would proabably need to open the link in Safari)

After that click in the lower right corner of the (possibly very wide) model viewer to start Augmented Reality.

## Add Flights in and out of Zurich Airport

Now let's grab some flights trajectories from [OpenSky Network](https://opensky-network.org). This is a network of ADS-B receivers around the glote. We have here some flights on Monday 2022-06-20 between 8am and 9am (UTC).

In [ ]:
flights = pd.read_csv("./data/flights_zrh.csv")
flights

Here for each flight (`callsign`) we have a timeseries (`time`) of different physical parameters: location, altitude, and velocity. We already have the lat and lon in a LV95-Version and the altitudes in meter, so we are set to plot the trajectories.

For this plot we use the following arguments:
* first argument is just the DataFrame, this works like e.g. in Plotly as we will see.
* `x`,`y`,`z`: are the coordinates in 3D space. We recenter and scale manually like in the surface plot.
* with the `col` parameter we set the color like in other plot libraries. This now specifices just the `callsign` column of the plots dataframe, i.e. `flights`. in a line plot this also gives the grouping so that PlotAR knows to what records belong to the same line.
* `type` gives the type of the plot, here we only want the line without scatter spheres.
* `width` is the width of the line.
* Finally `surface` here is the figure from the relief above - this just adds it to the same plot.

In [ ]:
_ = flights
plot = plotar.linear(
    _,
    x=(_.lon_lv95 - center[0]) * SCALE,
    y=(_.lat_lv95 - center[1]) * SCALE,
    z=(_.baroaltitude_m - ground) * SCALE * HEIGHT_FACTOR,
    auto_scale=False,
    col='callsign',
    col_labels=False, #there are too many to show
    type='l', width=0.5,
    name="Flights around Zurich over Relief",
    surface=fig,
)
plot

## Lorenz Attractor

The Lorenz system is a system of ordinary differential equations first studied by mathematician and meteorologist Edward Lorenz [[source: Wikipedia](https://en.wikipedia.org/wiki/Lorenz_system)]:
<math> \begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} &= \sigma (y - x), \\[6pt]
\frac{\mathrm{d}y}{\mathrm{d}t} &= x (\rho - z) - y, \\[6pt]
\frac{\mathrm{d}z}{\mathrm{d}t} &= x y - \beta z.
\end{align} </math>
> The equations relate the properties of a two-dimensional fluid layer uniformly warmed from below and cooled from above. In particular, the equations describe the rate of change of three quantities with respect to time: $x$ is proportional to the rate of convection, $y$ to the horizontal temperature variation, and $z$ to the vertical temperature variation. The constants σ, ρ, and β are system parameters proportional to the Prandtl number, Rayleigh number, and certain physical dimensions of the layer itself.

The Lorenz Attractor is a set of solutions to this system and a prominent example of chaotic behaviour.

We will not get into interpreting this too much, but just to show you something, that you might have seen in 2D before, in its natural habitat 3D.

Let's quickly calculate solutions:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0 / 3.0

def f(state, t):
    x, y, z = state  # Unpack the state vector
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z  # Derivatives

state0 = np.array([1.0, 1.0, 1.0])
t = np.arange(0.0, 40.0, 0.01)

states = odeint(f, state0, t)
states.shape

In [ ]:
plot = plotar.linear(states, auto_scale=True, type='l', name="lorenz")
plot

## OpenAI Embeddings of the SDS2024 events

Having more dimensions is actually very useful when you have to do dimension reduction: instead of reducing 1536 dimensions to a plane, better reduce it to space!

As an example we scraped all the events from <https://www.sds2024.ch> and will now use OpenAI embeddings for semantic similarity. After that we will perform dimension reduction with UMAP.

In a later example below there are also variants with calculating the embeddings with sentence-transformers and also with PCA and T-SNE as dimension reduction algorithms.

In [ ]:
import umap
import openai

In [ ]:
client = openai.AzureOpenAI()

In [ ]:
sds24 = pd.read_csv("./data/sds24.csv")
sds24

Now get the Embeddings from Azure OpenAI:

In [ ]:
response = client.embeddings.create(input=sds24.description.str.replace("\n",""), model="text-embedding-ada-002")

In [ ]:
f"{response.usage=} {len(response.data)=})"

Extract the embeddings as a numpy matrix:

In [ ]:
sds24_embeds = np.array([_.embedding for _ in response.data ])
sds24_embeds.shape

In [ ]:
def crop_text(x, max_characters):
    return pd.Series(x).map(lambda _: _ if len(_) <= max_characters-3 else _[:max_characters-3]+'...')

First for comparison we plot this using Plotly in 2 dimensions:

In [ ]:
# _ = pd.DataFrame(umap.UMAP(n_components=2, random_state=42, n_jobs=1).fit_transform(sds24_embeds), columns=['x','y'])
_ = sds24.copy()
_[['x','y']] = pd.DataFrame(umap.UMAP(n_components=2, random_state=42, n_jobs=1).fit_transform(sds24_embeds))
display(_.head())

px.scatter(
    _, 'x','y',
    text=crop_text(sds24.name, 15),
    color='format',
    height=500,
)

And now in 3D. We use PlotARs scatter plot with text-labels:
* first argument is the DataFrame we use.
* `xyz` specifies the coordinates in space for each entry
* `label` is the title of the event
* `col` specifies the color of the sphere and adds the legend to understand it
* we disable `axis_names` because they have not much meaning here

In [ ]:
_ = sds24.copy()
_[['x','y','z']] = pd.DataFrame(umap.UMAP(n_components=3, random_state=42, n_jobs=1).fit_transform(sds24_embeds), columns=['x','y','z'])
display(_.head())

plotar.plotar(
    _,
    xyz=['x','y','z'],
    label=crop_text(sds24.name, 30),
    col='format',
    axis_names=False,
)

# More examples to try out

## GAPminder

Let us look at the example on country development: **Gapminder**

This was made famouse by Hans Rosling's [Lecture](https://www.youtube.com/watch?v=jbkSRLYSojo) showing how world wars etc. and general development shaped countries by looking at ther income per capita, life expectancy and population as size.

In [ ]:
url = 'https://github.com/UofTCoders/2018-09-10-utoronto/raw/gh-pages/data/world-data-gapminder.csv'
gap = pd.read_csv(url)
gap

To keep it simple let's look only at some European countries:

In [ ]:
countries = [
 'Austria',
 'Switzerland',
 'Spain',
 'Poland',
 'Italy',
 'Greece',
 'Germany',
 'France',
]

In [ ]:
gap_subset = gap.query("country.isin(@countries)")
gap_subset = (
    gap_subset.groupby(['country',gap_subset.year.round(-1)])[['population', 'income','life_expectancy']]
    .mean().round().astype(int).reset_index()
)
gap_subset

We first plot the time series of *life expectancy* by *country* in a classical line plot:

In [ ]:
px.line(gap_subset, x='year', y='life_expectancy',
              color='country',)

What if we would like to add some more information? For instance the *income* and *population*?

In [ ]:
plot = plotar.linear(gap_subset, xyz=['year', 'income','life_expectancy'],
              col='country', size='population')
plot

## GAPminder animated

We can do animation in PlotAR!

In [ ]:
plot = plotar.animate(gap_subset, xyz=['year', 'income','life_expectancy'],
    group='country', col='country', size='population', animation_frame='year',
    name="gapminder-animated")
plot

The spheres are nice, however we don't know which dot is which country - so take the country name directly to the plot:

In [ ]:
plot = plotar.animate(gap.query("region=='Europe'"), xyz=['children_per_woman','income','life_expectancy'],
    group='country', col='sub_region', size='population', animation_frame='year',
    label = 'country', name="gapminder-animated-label")
plot

## D ONE Team

We scrape the http://d-one.ai/team webpage and extract some features on the team member's description

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = 'https://d-one.ai/team'
res = requests.get(url)

In [ ]:
pd.set_option("max_colwidth", 400)

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')
x = soup.find_all("div", class_="details")
team = pd.DataFrame( dict(name=_.find_all('h3')[0].text, text=_.find_all('p')[0].text) for _ in x )
team = team.drop_duplicates('name')
team

Extract two approximate counts: number of words and number of sentence (the latter actuall fails e.g. if many Abbreviations are used :-| )

In [ ]:
team['n_sent'] = team.text.str.replace(r'[^.]','', regex=True).str.len()
team['n_word'] = team.text.str.replace(r'[^ ]','', regex=True).str.len()+1

Extract the last mentioned year - usually that is, when people started. If we do not find one, take 2000 as a default value - that is before the company was founded!

In [ ]:
years = team.text.apply(lambda x: ([2000] + [_ for _ in x.split() if _.startswith("20")])[-1])
team['year_start'] = years.astype(str).str.rstrip('.').astype(int)
team['year_start'].value_counts(dropna=False)

In [ ]:
team['dr'] = team.name.str.startswith("Dr.")
team['dr'].value_counts(dropna=False)

In [ ]:
team

In [ ]:
plot = plotar.plotar(team, xyz=['n_word', 'n_sent', 'year_start'], col='dr',
    label='name')
plot

In [ ]:
plot.write("examples/d-one-team.json", format="json gltf glb usda usdz".split())

## Wikipedia Embeddings

In [ ]:
import json

In [ ]:
with open("data/metaverse_embeds.json") as f:
    metaverse = json.load(f)

In [ ]:
embed = pd.DataFrame(metaverse)
embed

In [ ]:
X = np.stack(embed.vector.to_list())
X.shape

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(n_components=3, random_state=42).fit_transform(X)
X_embedded.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
X_embedded = PCA(n_components=3, random_state=42).fit_transform(X)
X_embedded.shape

In [ ]:
import umap

In [ ]:
X_embedded =umap.UMAP(n_components=3, random_state=42).fit_transform(X)
X_embedded.shape

In [ ]:
plot = plotar.plotar(X_embedded, label=embed.title, size=embed.nword)
plot

In [ ]:
plot.write('examples/wikipedia-metaverse.json', format=["json","usdz","glb","html"])

In [ ]:
glove = pd.read_csv('data/glove-wiki-gigaword-50.gz',sep=' ', quotechar=";",
              skiprows=99,header=None,nrows=400, compression='gzip')
glove

In [ ]:
X_embedded =umap.UMAP(n_components=3, random_state=42).fit_transform(glove.iloc[:200,1:])
X_embedded.shape

In [ ]:
n = X_embedded.shape[0]
plot = plotar.plotar(X_embedded, label=glove[0][:n], size=1/(glove[:n].index+1))

In [ ]:
plot.write('examples/glove-wikipedia.json', format=["json","usdz","glb","html","gltf"])

In [ ]:
plot.write('examples/glove-wikipedia.json', format=["json","usdz","glb","html","gltf"])

In [ ]:
glove[0][90:110]

## CH - surface of Switzerland

We use the [Swisstopo Digital Height Model](https://www.swisstopo.admin.ch/de/geodata/height/dhm25200.html) 200m grid to draw a surface of Swizterland.

In [ ]:
url = 'https://data.geo.admin.ch/ch.swisstopo.digitales-hoehenmodell_25/data.zip'
file_name = 'DHM200.asc'

Download Zip file, unzip the part we need to file:

In [ ]:
def get_or_download(url, file_name, cache="tmp"):
    file = Path(cache) / file_name
    if not file.exists():
        from io import BytesIO
        from zipfile import ZipFile
        import shutil
        print(f"Downloading {url} to {file} ...")
        zipfile = ZipFile(BytesIO(requests.get(url).content))
        with open(file, 'wb') as f:
            shutil.copyfileobj(zipfile.open(file_name), f)
        print(f"Downloaded {file} from {url}")
    else:
        print(f"getting {file} from cache")
    return file

In [ ]:
file = get_or_download(url, file_name)

The GeoSpatial Information is in the first 6 rows of the file:

In [ ]:
%%time
y_head = {k: float(v) for k,v in np.genfromtxt(file, dtype=str, max_rows=6)}
print(y_head)
y = np.genfromtxt(file, skip_header=6, skip_footer=1)
y.shape

In [ ]:
n,m = [int(y_head[_]) for _ in ['NCOLS','NROWS']]
n,m

In [ ]:
img = y.flatten()[:n*(m-1)].reshape((m-1,n))

This file is actually rather big - if you want you can make it smaller by setting factor to e.g. 5, 10, 20. We set it to 2 for mybinder.org

In [ ]:
factor = 2

In [ ]:
factor = 4

In [ ]:
img = img[::factor,::factor]

In [ ]:
xvec = np.arange(img.shape[1]) * y_head['CELLSIZE'] * factor
yvec = np.arange(img.shape[0]) * y_head['CELLSIZE'] * factor

Impute negative, i.e. NA values to some level below switerlands elevation

In [ ]:
img[img>0].min()

In [ ]:
img[img<0] = 150

Quickly draw it here so we understand whats happening:

In [ ]:
plt.imshow(img, interpolation='none');

Actually - since the Swiss geographical coordinate system (LV95) is in meters (our xvec and yvec), and the height is as well - this is in all our export formats a correct scale representation of the surface of switerzland!

In the plots obviously it will be shown on a much smaller scale.

In [ ]:
plot = plotar.surfacevr(img, x=xvec, y=yvec, auto_scale=False, name="CH")
plot

## CH-color - surface of Switzerland with color

Now add the official satellite image on top of that surface.

In [ ]:
landsat_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "LandsatMos25.tif")
landsat_metadata_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "Landsatmos25.TFW")

In [ ]:
y_head

In [ ]:
sat_head = np.genfromtxt(landsat_metadata_file).astype(np.int64)
sat_head.T

Description [[Source](http://www.omg.unb.ca/~jonnyb/processing/geotiff_tifw_format.html)]:
* First row is x-pixel resolution
* Second and third rows are so-called "rotational components" but are set to zero in the case of an unrotated mapsheet.
* The fourth row is the y-pixel resolution. The negative sign indicates that the image y-axis is positive down which is the opposite from real world coordinates.
* The 5th and 6th rows are the Easting and Northing of the upper left pixel (0,0 in image coordinates). 

If you compare `y_head` and `sat_head` you see that unfortunately we need to crop the satellite to match the frame of the surface data:

In [ ]:
crop = (
    -sat_head[4] + (y_head['XLLCORNER']),
    sat_head[5] - (y_head['YLLCORNER'] + y_head['CELLSIZE'] * y_head['NROWS']),
)
crop = crop + (
    crop[0] + y_head['CELLSIZE'] * y_head['NCOLS'],
    crop[1] + y_head['CELLSIZE'] * y_head['NROWS'],
)
np.array(crop)/25.0

In [ ]:
from PIL import Image

In [ ]:
landsat = Image.open(landsat_file)

No crop it and rescale it to the size of the surface

In [ ]:
landsat_small = landsat.crop(np.array(crop)/25.0).resize(reversed(img.shape))

In [ ]:
landsat_small.size, np.array(landsat_small).shape, img.shape

In [ ]:
landsat_small

No plot it and resize it - also exaggerate the height by a factor ~3

In [ ]:
plot = plotar.surfacevr(img/100000, x=xvec/300000, y=yvec/300000, surfacecolor=np.array(landsat_small),
                             auto_scale=False, name="CH-color",)

In [ ]:
plot.write("examples/CH-color.json", format="json gltf glb usda usdz".split())

## Planets

We visualize the position of the Planets in the solar system at some time using the skyfield package.

In [ ]:
from skyfield.api import Loader
import io

In [ ]:
load = Loader("./tmp/")

In [ ]:
ts = load.timescale()
t = ts.utc(2022, 5, 22, 15, 19)
tarr = ts.utc(2022, 5, range(-365,365, 14))

**Note:** on mybinder.org unfortunately ftp-downloads are blocked so this will run into a timeout. We are preparing a workaround.

In [ ]:
planets = load('de421.bsp')  # ephemeris DE421

In [ ]:
planet_names = [ _[-1] for i,_ in planets.names().items() if 0 < i < 100 ]
print(len(planet_names))
planet_names

https://en.wikipedia.org/wiki/Planet

In [ ]:
_ = """i	Name	Equatorial diameter [i]	Mass [i]	Semi-major axis (AU)	Orbital period (years)	Inclination to Sun's equator (°)	Orbital eccentricity	Rotation period (days)	Confirmed moons	Axial tilt (°)	Rings	Atmosphere
1.	Mercury	0.383	0.06	0.39	0.24	3.38	0.206	58.65	0	0.10	no	minimal
2.	Venus	0.949	0.81	0.72	0.62	3.86	0.007	−243.02	0	177.30	no	CO2, N2
3.	Earth	1.000	1.00	1.00	1.00	7.25	0.017	1.00	1	23.44	no	N2, O2, Ar
4.	Mars	0.532	0.11	1.52	1.88	5.65	0.093	1.03	2	25.19	no	CO2, N2, Ar
5.	Jupiter	11.209	317.83	5.20	11.86	6.09	0.048	0.41	79	3.12	yes	H2, He
6.	Saturn	9.449	95.16	9.54	29.45	5.51	0.054	0.44	82	26.73	yes	H2, He
7.	Uranus	4.007	14.54	19.19	84.02	6.48	0.047	−0.72	27	97.86	yes	H2, He, CH4
8.	Neptune	3.883	17.15	30.07	164.79	6.43	0.009	0.67	14	29.60	yes	H2, He, CH4j"""
planet_info = pd.read_csv(io.StringIO(_), delimiter='\t').drop(columns=['i']).set_index("Name")
planet_info

In [ ]:
planet_info['Equatorial diameter [i]']

In [ ]:
planets_traj_xyz = pd.concat([
    pd.DataFrame(planets[_].at(tarr).ecliptic_xyz().au.T, columns=list('xyz'))
    .assign(planet=_).assign(t=tarr.utc_strftime())
    for _ in planet_names
])
planets_traj_xyz

In [ ]:
plot = plotar.animate(planets_traj_xyz, xyz=['x','y','z'],
              group='planet', col='planet', size=planet_info['Equatorial diameter [i]'].to_list()+[1,1],
              animation_frame='t', name='planets')
plot

In [ ]:
plot.write("examples/planets.json", format="json gltf glb usda usdz".split())

## Flights

In [ ]:
from skyfield.api import N, W, wgs84, load
from skyfield.functions import length_of

In [ ]:
url = 'https://www.flightradar24.com/flights/most-tracked'
# flightradar24 refuses 'User-Agent': 'python-requests/2.25.1' with error 451 Unavailable For Legal Reasons
res = requests.get(url, headers={'User-Agent': ''})

In [ ]:
most_tracked = pd.DataFrame(res.json()['data'])
most_tracked['name'] = most_tracked.fillna('_').apply(
    lambda _: f"{_.callsign} {_.from_city}->{_.to_city}", axis=1)
most_tracked

In [ ]:
def get_flight(flight_id, name):
    url = f'https://data-live.flightradar24.com/clickhandler/?version=1.5&flight={flight_id}'
    res = requests.get(url)
    trail = pd.DataFrame(res.json()['trail'])
    trail['flight_id'] = flight_id
#     trail['name'] = name
    return trail

In [ ]:
flights = pd.concat(( get_flight(_.flight_id, _.name) for _ in most_tracked.itertuples()), )

In [ ]:
trail = most_tracked.merge(flights, on="flight_id")
trail

In [ ]:
_ = trail.apply(lambda _: wgs84.latlon(_.lat, _.lng, _.alt*10).at(t).position.m, axis=1)
trail[['x','y','z']] = np.stack(_) / 1000
trail

In [ ]:
plot = plotar.linear(trail, xyz=['x','y','z'], col='name', size=trail.spd/10, auto_scale=True, type='l', name='flights')
plot

## SOLA 2022

The [Sola 2022](https://trackmaxx.ch/maps/?m=ec368d93-aff4-4a7e-b0a5-24b7f9683a32&style=swisstopo&legend=full&tracks=1,2,3,4,5,6,8,7,9,10,11,12,13,14,20&labels=iconsubergabebuchlern,iconsuebergaben,icstrecke10,icstrecke11,icstrecke14,icstrecke2,icstrecke3,icstrecke4,icstrecke5,icstrecke6,icstrecke7,icstrecke8,icugbucheggplatz,icstrecke9,icugegg,icugfelsenegg,icugfluntern,icugforch,icughoenggerberg,icugirchel,icuguetliberg,icugwitikon,icugzumikon&h=8d68)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = "https://tmxx-static.s3.amazonaws.com/ous/asvzsolazh/mapstudio/gpx/strecke01.gpx"
res = requests.get(url)

In [ ]:
pd.set_option("max_colwidth", 400)

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
def sola_track(strecke):
    url = f"https://tmxx-static.s3.amazonaws.com/ous/asvzsolazh/mapstudio/gpx/strecke{strecke:02}.gpx"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    x = soup.find_all("trkpt")
    track = pd.DataFrame( _.attrs for _ in x )
    track['ele'] = pd.Series( _.ele.text for _ in x)
    track = track.astype(float)
    track['strecke'] = strecke
    return track
sola_track(1)

In [ ]:
sola = pd.concat( (sola_track(_) for _ in range(1,15)), axis=0)
sola

In [ ]:
sola.plot.line('lon','lat',);

In [ ]:
plot = plotar.linear(sola, xyz=['lon','lat','ele'], col=sola.strecke.astype(str), auto_scale=True, type='l', name="sola")
plot

## Eyetracking

<https://www.eyetracking-eeg.org/testdata.html>

In [ ]:
eyetracking_file = get_or_download("https://www.eyetracking-eeg.org/testdata/freeviewing.zip", "eyetracker_freeviewing.txt")

In [ ]:
url = 'https://www.eyetracking-eeg.org/testdata/reading.zip'

In [ ]:
eyetracking = pd.read_table('tmp/eyetracker_freeviewing.txt',
    skiprows=20, header=None,
    names=['Time', 'Type', 'Trial', 'L Dia [mm]', 'L Area [mm]', 'R Dia [mm]', 'R Area [mm]', 'L POR X [px]', 'L POR Y [px]', 'R POR X [px]', 'R POR Y [px]', 'Trigger']
)
eyetracking

In [ ]:
triggertypes = {
    103: 'Start-event for synchronization',
    12: 'Onset of a picture on the screen',
    1: 'Onset of search target (grey disc) within picture',
    99: 'Response: participant found the target and pressed a button.',
    203: 'End-event for synchronization',
}

In [ ]:
eyetracking['Trigger'] = eyetracking.Trigger.map(triggertypes).fillna('')

In [ ]:
eyetracking.head(10000).Trigger.value_counts()

In [ ]:
eyetracking['group'] = (eyetracking.Trigger == eyetracking.Trigger.shift(1)).cumsum()

In [ ]:
eyetracking.head(10000).query("Trigger != ''")

In [ ]:
plot = plotar.linear(eyetracking.head(10000), xyz=['L POR X [px]', 'Time', 'L POR Y [px]'],
    col='Trigger', #group='group',
    type='l', width=0.1)
plot

![image](https://www.eyetracking-eeg.org/images/testdata/testdata_search.jpg)

In [ ]:
plot.write('examples/eyetracking.json', format=['json','html','glb','usdz'])

# Wrap-up

Thanks a lot for trying PlotAR out - we hope we could open your mind with these experiences.